In [15]:
# example of tending the vgg16 model
import tensorflow as tf
from tensorflow import keras
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
import numpy as np

In [2]:
tf.__version__

'2.3.0'

In [3]:
keras.__version__

'2.4.0'

In [4]:
img_height=224
img_width=291
batch_size=32

In [5]:
train_ds=tf.keras.preprocessing.image_dataset_from_directory(
            '../data_grayscale/', labels='inferred', label_mode='int',
            validation_split=0.2, subset='training', seed=123,
#             color_mode='grayscale', 
            image_size=(img_height, img_width),
            batch_size=batch_size)

Found 671 files belonging to 2 classes.
Using 537 files for training.


In [6]:
val_ds=tf.keras.preprocessing.image_dataset_from_directory(
            '../data_grayscale/', labels='inferred', label_mode='int',
            validation_split=0.2, subset='validation', seed=123,
#             color_mode='grayscale', 
            image_size=(img_height, img_width),
            batch_size=batch_size)

Found 671 files belonging to 2 classes.
Using 134 files for validation.


In [7]:
class_names=train_ds.class_names
print(class_names)

['par1', 'wt']


In [8]:
base_model = keras.applications.Xception(weights='imagenet', 
                                        input_shape=(img_height,img_width,3),
                                        include_top=False)

In [9]:
base_model.trainable

True

In [10]:
base_model.trainable = False
base_model.trainable

False

In [16]:
inputs = keras.Input(shape=(img_height,img_width,3))

norm_layer = keras.layers.experimental.preprocessing.Normalization()
mean=np.array([127.5]*3)
var = mean**2
x = norm_layer(inputs)
norm_layer.set_weights([mean,var])

x=base_model(x, training=False)
x=keras.layers.GlobalAveragePooling2D()(x)
outputs=keras.layers.Dense(1)(x)
model=keras.Model(inputs,outputs)

In [17]:
model.compile(optimizer=keras.optimizers.Adam(),
             loss=keras.losses.BinaryCrossentropy(from_logits=True),
             metrics=[keras.metrics.BinaryAccuracy()])
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 291, 3)]     0         
_________________________________________________________________
normalization_1 (Normalizati (None, 224, 291, 3)       7         
_________________________________________________________________
xception (Functional)        (None, 7, 9, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2049      
Total params: 20,863,536
Trainable params: 2,049
Non-trainable params: 20,861,487
_________________________________________________________________


In [18]:
epochs=10
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/10
17/17 [==============================] - 7s 415ms/step - loss: 0.3656 - binary_accuracy: 0.8864 - val_loss: 0.1513 - val_binary_accuracy: 1.0000
Epoch 2/10
17/17 [==============================] - 7s 394ms/step - loss: 0.0918 - binary_accuracy: 1.0000 - val_loss: 0.0521 - val_binary_accuracy: 1.0000
Epoch 3/10
17/17 [==============================] - 7s 393ms/step - loss: 0.0385 - binary_accuracy: 1.0000 - val_loss: 0.0287 - val_binary_accuracy: 1.0000
Epoch 4/10
17/17 [==============================] - 7s 385ms/step - loss: 0.0239 - binary_accuracy: 1.0000 - val_loss: 0.0198 - val_binary_accuracy: 1.0000
Epoch 5/10
17/17 [==============================] - 7s 395ms/step - loss: 0.0173 - binary_accuracy: 1.0000 - val_loss: 0.0152 - val_binary_accuracy: 1.0000
Epoch 6/10
17/17 [==============================] - 7s 385ms/step - loss: 0.0136 - binary_accuracy: 1.0000 - val_loss: 0.0122 - val_binary_accuracy: 1.0000
Epoch 7/10
17/17 [==============================] - 7s 385ms/ste